In [ ]:
import numpy as np
import pandas as pd
from functools import partial
import glob
# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter
# from geopy import Point
from geopy import distance
from datetime import date, datetime
# from pymongo import MongoClient
from tqdm.auto import tqdm
from funcy import decorator
import holoviews as hv
import panel as pn
from configparser import ConfigParser
import plotly.graph_objs as go
import asyncio
import asyncpg
import aiosql
import nest_asyncio
import gc
import matplotlib.pyplot as plt
from bokeh.models import HoverTool
from plotly_calplot import calplot
import geopandas as gpd
import shapely
# import leafmap.foliumap as leafmap
# import leafmap
import folium
shapely.speedups.disable()
gc.enable()
nest_asyncio.apply()
hv.extension('bokeh', 'plotly',logo=False)
pn.extension('ipywidgets')

In [ ]:
input_path = 'data/cyclistic/raw'
files_format = "csv"
original_data_combined = "data/cyclistic/cyclistic_last_12_months_trips.csv"
processed_data = "data/cyclistic/cyclistic_last_12_months_trips_processed.csv"
original_data_table_name = 'cyclistic'
temp_csv_postgres = "'/tmp/cyclistic.csv'"
driver = "asyncpg"
temp_csv = '/tmp/cyclistic.csv'
sql_path = "https://raw.githubusercontent.com/adiladawi/holoviz/main/data/cyclistic/aiosql_cyclistic.sql"
reversed_gps_data = 'data/cyclistic/cyclistic_last_12_months_trips_end_reveresed_gps.csv'
sample_data = "https://raw.githubusercontent.com/adiladawi/holoviz/main/data/cyclistic/sample.csv"
processed_df_path = 'https://raw.githubusercontent.com/adiladawi/holoviz/main/data/cyclistic/cyclistic_processed.csv'
chicago_map = 'https://raw.githubusercontent.com/adiladawi/holoviz/main/data/cyclistic/chicago.geojson'
log_file = "logs/logfile.txt"
slides_file = 'https://raw.githubusercontent.com/adiladawi/holoviz/main/assets/Data_Analysis.slides.html'

# Analysis Section

### Introduction
In 2016, Cyclistic launched a successful bike-share offering. Since then, the program has grown to a fleet of 5,824 bicycles that are geotracked and locked into a network of 692 stations across Chicago. The bikes can be unlocked from one station and returned to any other station in the system anytime. Cyclistic sets itself apart by also offering reclining bikes, hand tricycles, and cargo bikes, making bike-share more inclusive to people with disabilities and riders who can’t use a standard two-wheeled bike. The majority of riders opt for traditional bikes; about 8% of riders use the assistive options. Cyclistic users are more likely to ride for leisure, but about 30% use them to commute to work each day.

Until now, Cyclistic’s marketing strategy relied on building general awareness and appealing to broad consumer segments. One approach that helped make these things possible was the flexibility of its pricing plans: single-ride passes, full-day passes, and annual memberships. Customers who purchase single-ride or full-day passes are referred to as casual riders. Customers who purchase annual memberships are Cyclistic members. Cyclistic’s finance analysts have concluded that annual members are much more profitable than casual riders. Although the pricing flexibility helps Cyclistic attract more customers, The Director of Marketing Moreno believes that maximizing the number of annual members will be key to future growth. Rather than creating a marketing campaign that targets all-new customers, Moreno believes there is a very good chance to convert casual riders into members. She notes that casual riders are already aware of the Cyclistic program and have chosen Cyclistic for their mobility needs.

Moreno has set a clear goal: Design marketing strategies aimed at converting casual riders into annual members. In order to do that, however, the marketing analyst team needs to better understand:

1- how annual members and casual riders differ

2- why casual riders would buy a membership

3- how digital media could affect their marketing tactics

Moreno and her team are interested in analyzing the Cyclistic historical bike trip data to identify trends.
Your are a junior data analyst working in the marketing analyst team. **Moreno has assigned you the first question to answer: How do annual members and casual riders use Cyclistic bikes differently?** To answer that, you are going to conduct this data analysis. From the analysis insights, marketing team will design a new marketing strategy to convert casual riders into annual members. But first, Cyclistic executives must approve marketing team recommendations, so the analysis must be backed up with compelling data insights and professional data visualizations

### Table of Contents  
<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#section1">Main Objective</a>  
2. <a href="#section2">Data Acquisition</a>  
3. <a href="#section3">Data Processing</a>  
4. <a href="#section4">Exploratory Data Analysis</a>  
5. <a href="#section5">Key Findings</a>  
6. <a href="#section6">Recommendations</a>  

</font>
</div>

### Main Objective

| Item |Defintion|
| :-- | :-- |
| Category Of Data Analysis | Descriptive | 
| Business Task Need To Be Addressed| Identify how annual members and casual riders use cyclistic bikes differently | 
| Goal Of The Business Task | Help marketing team in designing marketing strategies aimed at converting casual riders into annual members | 
| KPI | Members Conversion Rate | 
| Key Stakeholders | Cyclistic Executive Team, Director of Marketing (Lily Moreno) and Marketing Analytics Team | 

### Data Acquisition


Cyclistic’s historical trip data has been made available by [Motivate International Inc](https://www.motivateco.com/). under this [license](https://www.divvybikes.com/data-license-agreement)

[Data Source](https://divvy-tripdata.s3.amazonaws.com/index.html)

The portion of the data that is required to be analyzed is the previous 12 months of Cyclistic trip data Data. the information resides in ZIP files, so they need extracting first. the resulting is 12 CSV files format with 13 columns in each one,

| No. | Column | Description| Data Type | Category|
| :-- | :-- | :-- | :-- | :-- |
| 1 | Ride_Id | Customer Unique ID | String | Nominal |
| 2 | Rideable_Type | The type of bike customer used for the trip | String | Nominal |
| 3 | Started_At | The date and time when the trip started | DateTime | Discrete |
| 4 | Ended_At | The date and time when the trip ended | DateTime | Discrete |
| 5 | Start_Station_Name | Name of the station where the trip started | String | Nominal |
| 6 | Start_Station_Id | ID of the station where the trip started | String | Nominal |
| 7 | End_Station_Name | Name of the station where the trip ended | String | Nominal |
| 8 | End_Station_Id | ID of the station where the trip ended | String | Nominal |
| 9 | Start_Lat | Latitude value where the trip have started expressed in decimal degrees | Float | Contiuous |
| 10 | Start_Lng | Longitude value where the trip have started expressed in decimal degrees | Float | Contiuous |
| 11 | End_Lat | Latitude value where the trip have ended expressed in decimal degrees | Float | Contiuous |
| 12 | End_Lng | Longitude value where the trip have ended expressed in decimal degrees | Float | Contiuous |
| 13 | Member_Casual | Classification of Customers: Customers who purchase single-ride or full-day passes are referred to as casual. Customers who purchase annual memberships referred to as members  | String | Nominal |




Since the dataset is made public by a known corporation and has a license of how we can use it, we can safely say the data is credible.

The Longitude and latitude information have been extracted, enriched with Suburb information, and provided as separate CSV files to join them with original data later on.

Also, map file will be downloaded from the [City of Chigaco Website](https://data.cityofchicago.org/api/geospatial/cauq-8yn6?method=export&format=GeoJSON), which will help in producing geographic visualization

### Data Processing

Defining functions to process the data

In [ ]:
def log(message):
    '''Log function to write the time and custom message to the log file'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S'  # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now()  # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(log_file, "a") as f:
        f.write(timestamp + ',' + message + '\n')

In [ ]:
log("Cyclistic Notebook Execution Started")

In [ ]:
@decorator
def timing_log(call):
    '''Log function to write message and the time when certain function started or ended'''
    start_time = datetime.now()  # get current timestamp
    log('Function ' + call._func.__name__ + ' Started')
    result = call()
    end_time = datetime.now()
    log('Function ' + call._func.__name__ + ' Ended')
    duration = end_time - start_time
    log('Function ' + call._func.__name__ + ',' + 'took ' + str(duration.seconds) + ' seconds to run')
    return result

In [ ]:
@timing_log
def extract(file_to_process):
    '''Read csv file into Pandas DataFrame'''
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [ ]:
@timing_log
def combine_from_csv(directory, extension='csv'):
    '''Read multiple files under one directory, having the same file type into Pandas DataFrame'''
    combined_data = pd.DataFrame()
    for file in tqdm(glob.glob(f'{directory}/*.{extension}')):
        combined_data = combined_data.append(read_from_csv(file), ignore_index=True)
    return combined_data

In [ ]:
@timing_log
def extract_dir(*args, **kwargs):
    '''Read multiple files under one directory, having the same file type into Pandas DataFrame'''
    extracted_data = pd.DataFrame()
    extracted_data = extracted_data.append(combine_from_csv(*args, **kwargs))
    return extracted_data

In [ ]:
@timing_log
def load(data_to_load, targetfile):
    '''Write csv file from Pandas DataFrame'''
    data_to_load.to_csv(targetfile, index=False)

In [ ]:
log("ETL Process Started")

Extracting The Data

In [ ]:
log("Data Extract Phase Started")
df = extract(processed_df_path)
log("Data Extract Phase Ended")

In [ ]:
log("SQL Queris Loading for Transform Started")
pg_queries = aiosql.from_path(sql_path, driver)
log("SQL Queris Loading for Transform Ended")
log("Reverse Geocoding Data Loading for Transform Started")
reversed_data = extract(reversed_gps_data)
log("Reverse Geocoding Data Loading for Transform Ended")

previewing the data

In [ ]:
df.tail(n=2)

Summary of columns with missing values

In [ ]:
@timing_log
def missing_chart(df):
    '''Function to draw horizontal bar chart for columns with missing values'''
    missing = round((df.isnull().sum() / df.shape[0]) * 100, 2)
    missing = missing[missing > 0]  # filtered series
    missing = pd.DataFrame({'Column': missing.index, 'Percentage': missing.values})
    bar = hv.Bars(missing).sort(by='Percentage').opts(xrotation=35, invert_axes=True)
    labels = hv.Labels(missing,  kdims=['Column', 'Percentage'],
                       vdims='Percentage').opts(text_font_size='10pt')
    return (bar * labels).opts(width=1000)

In [ ]:
# missing_chart(df)

In [ ]:
@timing_log
def get_postgres_params(filename='config/config_file.ini', section='postgresql'):
    '''Function to read login information from configuration file'''
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
    # get section, default to postgresql
    db_params = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db_params[param[0]] = param[1]
    return db_params

In [ ]:
@timing_log
async def connect_to_postgres():
    '''Function to connect to Postgres Database'''
    db_params = get_postgres_params()
    conn = await asyncpg.connect(**db_params)
    return conn

In [ ]:
@timing_log
async def write_csv_to_postgres(csv_path, table_name):
    '''Function to read csv data into existing Postgres Database table'''
    conn = await connect_to_postgres()
    await conn.copy_to_table(table_name, source=csv_path, format='csv', delimiter=',', null="", header=1)
    await conn.close()

In [ ]:
@timing_log
async def write_df_to_postgres(df, table_name):
    '''Function to write Pandas DataFrame into Postgres Database table'''
    conn = await connect_to_postgres()
    table_drop_sql = f"DROP TABLE IF EXISTS {table_name};"
    await conn.execute(table_drop_sql)
    sql_create = pd.io.sql.get_schema(df, table_name)
    await conn.execute(sql_create)
    tempcsv = f'{table_name}.csv'
    df.to_csv(tempcsv, index=False)
    await conn.copy_to_table(table_name, source=tempcsv, format='csv', delimiter=',', null="", header=1)
    await conn.close()

In [ ]:
@timing_log
async def ussi():
    '''Helper function to update the missing data'''
    swui = df[df['start_station_id'].notna()]
    swui = df.groupby(['start_station_id', 'start_lat', 'start_lng']).size().reset_index(name='count')
    swmi = df[df['start_station_id'].isna()]
    swmi = df.groupby(['start_station_id', 'start_lat', 'start_lng']).size().reset_index(name='count')
    found_s_id_df = pd.merge(swui, swmi, how='left', on=["start_lat", "start_lng"])
    found_s_id_df = found_s_id_df[found_s_id_df['count_y'] > 0]
    found_s_id_df = found_s_id_df[["start_lat", "start_lng", "start_station_id_x"]]
    asyncio.get_event_loop().run_until_complete(write_df_to_postgres(found_s_id_df, 'temp_tbl'))
    conn = await connect_to_postgres()
    await pg_queries.ussi(conn)
    await conn.close()
    del swui, swmi, found_s_id_df

In [ ]:
@timing_log
async def ussi2():
    '''Helper function to update the missing data'''
    swui = df[(df['start_station_id'].notna()) & (df["start_station_name"].notna())]
    swui = df.groupby(['start_station_id', 'start_station_name']).size().reset_index(name='count')
    swmi = df[df['start_station_id'].isna() & (df["start_station_name"].notna())]
    swmi = df.groupby(['start_station_id', 'start_station_name']).size().reset_index(name='count')
    found_s_id_df = pd.merge(swui, swmi, how='left', on=["start_station_name"])
    found_s_id_df = found_s_id_df[found_s_id_df['count_y'] > 0]
    found_s_id_df = found_s_id_df[["start_station_name", "start_station_id_x"]]
    asyncio.get_event_loop().run_until_complete(write_df_to_postgres(found_s_id_df, 'temp_tbl'))
    conn = await connect_to_postgres()
    await pg_queries.ussi2(conn)
    await conn.close()
    del swui, swmi, found_s_id_df

In [ ]:
@timing_log
async def uesi():
    '''Helper function to update the missing data'''
    swui = df[df['end_station_id'].notna()]
    swui = df.groupby(['end_station_id', 'end_lat', 'end_lng']).size().reset_index(name='count')
    swmi = df[df['end_station_id'].isna()]
    swmi = df.groupby(['end_station_id', 'end_lat', 'end_lng']).size().reset_index(name='count')
    found_s_id_df = pd.merge(swui, swmi, how='left', on=["end_lat", "end_lng"])
    found_s_id_df = found_s_id_df[found_s_id_df['count_y'] > 0]
    found_s_id_df = found_s_id_df[["end_lat", "end_lng", "end_station_id_x"]]
    asyncio.get_event_loop().run_until_complete(write_df_to_postgres(found_s_id_df, 'temp_tbl'))
    conn = await connect_to_postgres()
    await pg_queries.uesi(conn)
    await conn.close()
    del swui, swmi, found_s_id_df

In [ ]:
@timing_log
async def uesi2():
    '''Helper function to update the missing data'''
    swui = df[(df['end_station_id'].notna()) & (df["end_station_name"].notna())]
    swui = df.groupby(['end_station_id', 'end_station_name']).size().reset_index(name='count')
    swmi = df[df['end_station_id'].isna() & (df["end_station_name"].notna())]
    swmi = df.groupby(['end_station_id', 'end_station_name']).size().reset_index(name='count')
    found_s_id_df = pd.merge(swui, swmi, how='left', on=["end_station_name"])
    found_s_id_df = found_s_id_df[found_s_id_df['count_y'] > 0]
    found_s_id_df = found_s_id_df[["end_station_name", "end_station_id_x"]]
    asyncio.get_event_loop().run_until_complete(write_df_to_postgres(found_s_id_df, 'temp_tbl'))
    conn = await connect_to_postgres()
    await pg_queries.uesi2(conn)
    await conn.close()
    del swui, swmi, found_s_id_df

In [ ]:
@timing_log
async def ussn():
    '''Helper function to update the missing data'''
    swui = df[(df["start_station_name"].notna()) & (df['start_station_id'].notna())]
    swui = df.groupby(['start_station_name', 'start_station_id']).size().reset_index(name='count')
    swmi = df[df["start_station_name"].isna() & (df['start_station_id'].notna())]
    swmi = df.groupby(['start_station_name', 'start_station_id',]).size().reset_index(name='count')
    found_s_id_df = pd.merge(swui, swmi, how='left', on=['start_station_id'])
    found_s_id_df = found_s_id_df[found_s_id_df['count_y'] > 0]
    found_s_id_df = found_s_id_df[["start_station_name_x", "start_station_id"]]
    asyncio.get_event_loop().run_until_complete(write_df_to_postgres(found_s_id_df, 'temp_tbl'))
    conn = await connect_to_postgres()
    await pg_queries.ussn(conn)
    await conn.close()
    del swui, swmi, found_s_id_df

In [ ]:
@timing_log
async def uesn():
    '''Helper function to update the missing data'''
    swui = df[(df["end_station_name"].notna()) & (df['end_station_id'].notna())]
    swui = df.groupby(['end_station_name', 'end_station_id']).size().reset_index(name='count')
    swmi = df[df["end_station_name"].isna() & (df['end_station_id'].notna())]
    swmi = df.groupby(['end_station_name', 'end_station_id',]).size().reset_index(name='count')
    found_s_id_df = pd.merge(swui, swmi, how='left', on=['end_station_id'])
    found_s_id_df = found_s_id_df[found_s_id_df['count_y'] > 0]
    found_s_id_df = found_s_id_df[["end_station_name_x", "end_station_id"]]
    asyncio.get_event_loop().run_until_complete(write_df_to_postgres(found_s_id_df, 'temp_tbl'))
    conn = await connect_to_postgres()
    await pg_queries.uesn(conn)
    await conn.close()
    del swui, swmi, found_s_id_df

In [ ]:
@timing_log
def update_id_and_name():
    '''Helper function to update the missing data'''
    asyncio.get_event_loop().run_until_complete(ussi())
    asyncio.get_event_loop().run_until_complete(ussi2())
    asyncio.get_event_loop().run_until_complete(uesi())
    asyncio.get_event_loop().run_until_complete(uesi2())
    asyncio.get_event_loop().run_until_complete(ussn())
    asyncio.get_event_loop().run_until_complete(uesn())

In [ ]:
@timing_log
def append_sql_query():
    '''Helper function to help write SQL query with qoutation marks in it to SQL script file'''
    query = f"COPY {original_data_table_name} TO {temp_csv_postgres} WITH (FORMAT CSV, HEADER);"
    string = f"""
-- name: copy_table_to_csv#
-- copy postgres table to csv file
{query}
"""
    with open(sql_path, 'a') as f:
        f.write(string)

In [ ]:
@timing_log
async def dump_table_to_csv():
    '''Function to write Postgres SQL table to csv file'''
    conn = await connect_to_postgres()
    await pg_queries.copy_table_to_csv(conn)
    await conn.close()

In [ ]:
@timing_log
def get_season(dates):
    '''Function to return the season for list or series of dates'''
    WS = date(2016, 12, 15)  # Winter Solstice
    SE = date(2016, 3, 15)  # Spring Equinox
    SS = date(2016, 6, 15)  # Summer Solstice
    FE = date(2016, 9, 15)  # Fall Equinox

    # Convert dates from any year to 2016 dates: leap year
    dates = [date.replace(year=2016) for date in dates]
    seasons = []
    for dt in dates:
        if (dt.date() >= WS) | (dt.date() < SE):
            seasons.append("Winter")
        elif (dt.date() >= SE) & (dt.date() < SS):
            seasons.append("Spring")
        elif (dt.date() >= SS) & (dt.date() < FE):
            seasons.append("Summer")
        else:
            seasons.append("Fall")
    return seasons

In [ ]:
@timing_log
async def update_suburb():
    '''Helper function to update the missing data'''
    conn = await connect_to_postgres()
    await pg_queries.uss(conn)
    await pg_queries.uss2(conn)
    await pg_queries.ues(conn)
    await pg_queries.ues2(conn)
    await conn.close()

Transform function

In [ ]:
@timing_log
def transform(df):
    '''Transform Function in the ETL process, which produce data that error free'''
    # uploding dataframe to postgres database
    asyncio.get_event_loop().run_until_complete(write_df_to_postgres(df, original_data_table_name))
    # Imputating missing data with joins
    update_id_and_name()
    # adding sql query to the external file to write query with qoutations
    append_sql_query()
    # reading sql queries from external file
    global pg_queries
    pg_queries = aiosql.from_path(sql_path, driver)
    # writing sql table to csv file
    asyncio.get_event_loop().run_until_complete(dump_table_to_csv())
    # reading the written csv file
    df = extract(temp_csv)
    # dropping null values
    df = df.dropna()
    # dropping duplicates
    df = df.drop_duplicates()
    # dropping quality rides from data
    df = df[df['start_station_name'] != "HQ QR"]
    # selecting numeric columns
    numeric = df.select_dtypes(include=[np.number]).columns
    # rounding numeric columns to 4 digits
    df[numeric] = np.round(df[numeric], 4)
    # creating location columns for reverse geocoding
    df['start_point'] = df['start_lat'].astype(str) + ',' + df['start_lng'].astype(str)
    df['end_point'] = df['end_lat'].astype(str) + ',' + df['end_lng'].astype(str)
    # creating empty columns to be filled with join opration in SQL
    df['start_suburb'] = pd.NA
    df['end_suburb'] = pd.NA
    # reading reversed geocoded data
    global reversed_data
    # upload dataframes to postgres
    asyncio.get_event_loop().run_until_complete(write_df_to_postgres(df, 'cyclistic'))
    asyncio.get_event_loop().run_until_complete(write_df_to_postgres(reversed_data, 'reveresed_tbl'))
    # performing the join operation in SQL
    asyncio.get_event_loop().run_until_complete(update_suburb())
    # writing updated sql table to csv file
    asyncio.get_event_loop().run_until_complete(dump_table_to_csv())
    # reading the written csv file
    df = extract(temp_csv)
    # dropping null values
    df = df.dropna()
    # casting columns to the correct type
    df[["started_at", "ended_at"]] = df[["started_at", "ended_at"]].apply(pd.to_datetime)
    df['rideable_type'] = df['rideable_type'].astype("category")
    df['member_casual'] = df['member_casual'].astype("category")
    # enriching data with new columns
    df['ride_length_mins'] = (df['ended_at'] - df['started_at']) / pd.Timedelta(minutes=1)
    df['ride_distance_km'] = [distance.distance(p1, p2).kilometers for p1, p2 in df[['start_point', 'end_point']].values]
    df['ride_speed_kmh'] = df['ride_distance_km'] / (df['ride_length_mins'] / 60)
    df['season'] = get_season(df['started_at'])
    df['ride_at_weekend'] = ["Yes" if day.weekday() < 5 else "No" for day in df['started_at']]
    df['date'] = df['started_at'].dt.date
    df['year_month'] = df['started_at'].dt.strftime('%Y-%m')
    df['year_month'] = df['year_month'].astype('category')
    bins = [0, 6, 12, 18, 23]
    labels = ["night", "morning", "afternoon", "evening"]
    df['time_of_day'] = pd.cut(df['started_at'].dt.hour, bins=bins, labels=labels, include_lowest=True)
    df['time_of_day'] = df['time_of_day'].astype('category')
    # casting columns to the correct type and modifying data for spatial join
    df['start_suburb'] = df['start_suburb'].astype('str')
    df['end_suburb'] = df['end_suburb'].astype('str')
    df['start_suburb'] = df['start_suburb'].map(str.upper)
    df['end_suburb'] = df['end_suburb'].map(str.upper)
    # quality check for the data
    df = df[df['ride_length_mins'] > 2]
    df = df[df['ride_length_mins'] < 720]
    df = df[df['ride_speed_kmh'] < 80]
    df = df[df['ride_distance_km'] < 40]
    return df

In [ ]:
# log("Data Transform Phase Started")
# df = transform(df)
# log("Data Transform Phase Ended")

In [ ]:
# log("Data Load Phase Started")
# load(df, processed_df_path)
# log("Data Load Phase Ended")

In [ ]:
log("ETL Process Ended")

Description of the data final form

| No. | Column | Description| Data Type | Category|
| :-- | :-- | :-- | :-- | :-- |
| 1 | Ride_Id | Customer Unique ID | String | Nominal |
| 2 | Rideable_Type | The type of bike customer used for the trip | String | Nominal |
| 3 | Started_At | The date and time when the trip started | DateTime | Discrete |
| 4 | Ended_At | The date and time when the trip ended | DateTime | Discrete |
| 5 | Start_Station_Name | Name of the station where the trip started | String | Nominal |
| 6 | Start_Station_Id | ID of the station where the trip started | String | Nominal |
| 7 | End_Station_Name | Name of the station where the trip ended | String | Nominal |
| 8 | End_Station_Id | ID of the station where the trip ended | String | Nominal |
| 9 | Start_Lat | Latitude value where the trip have started expressed in decimal degrees | Float | Contiuous |
| 10 | Start_Lng | Longitude value where the trip have started expressed in decimal degrees | Float | Contiuous |
| 11 | End_Lat | Latitude value where the trip have ended expressed in decimal degrees | Float | Contiuous |
| 12 | End_Lng | Longitude value where the trip have ended expressed in decimal degrees | Float | Contiuous |
| 13 | Member_Casual | Classification of Customers: Customers who purchase single-ride or full-day passes are referred to as casual. Customers who purchase annual memberships referred to as members  | String | Nominal |
| 14 | Month | The month when the trip happened  | Integer | Discrete |
| 15 | Start_Point | Latitude and Longitude of the starting station combined | String | Nominal |
| 16 | End_Point | Latitude and Longitude of the Ending station combined  | String | Nominal |
| 17 | Ride_Length_Mins | Duration of the ride measured in minutes | Float | Contiuous |
| 18 | Ride_Distance_KM | Distance of the ride measured in kilometers | Float | Contiuous |
| 19 | Ride_Speed_KMH | Speed of the ride measured in kilometers  per hour| Float | Contiuous |
| 20 | Season | Season at which the ride took place | String | Nominal |
| 21 | Ride_At_Weekend | Identification if the ride conducted during the weekend | String | Nominal |
| 22 | Start_Suburb | Starting suburb  of the ride  | String | Nominal |
| 23 | End_Suburb | Ending suburb  of the ride  | String | Nominal |
| 24 | Date | The date when the trip happened | Date | Discrete |
| 25 | Year_Month | The month and year when the trip happened | String | Nominal |
| 26 | Time_Of_Day | The date and time when the trip Occured | String | Nominal |

### Exploratory Data Analysis

In [ ]:
nums = ['ride_length_mins', 'ride_distance_km', 'ride_speed_kmh']
cats = ['rideable_type', 'season', 'ride_at_weekend', 'start_suburb', 'end_suburb', 'time_of_day', 'year_month']
chartable = [x for x in cats if df[x].nunique() < 6]
dates = ['date']

**Descriptive Analysis Of Numerical Fields (Table 1)**

In [ ]:
def desc_num(df, group_col, nums):
    q_25 = partial(pd.Series.quantile, q=0.25)
    q_25.__name__ = "Q1"
    q_75 = partial(pd.Series.quantile, q=0.75)
    q_75.__name__ = "Q3"
    data_range = partial(np.ptp)
    data_range.__name__ = "range"
    nums = df.select_dtypes(include='number').columns.tolist() if nums is None else nums
    grouped = df.groupby(group_col)[nums].agg(["count", "min", q_25, "median", "mean", q_75, "max", data_range, "var", "std"])
    grouped = round(grouped.T.reset_index(), 1)
    grouped.columns = ['Column', 'Measure'] + df[group_col].map(str.upper).unique().tolist()
    return grouped

In [ ]:
desc_num(df, "member_casual", nums).head(10)

**Descriptive Analysis Of Categorical Fields (Table 2)**

In [ ]:
def desc_cat(df, group_col, cats):
    cats = df.select_dtypes(exclude='number').columns.tolist() if nums is None else cats
    grouped = df.groupby(group_col)[cats].describe()
    grouped = grouped.T.reset_index()
    grouped.columns = ['Column', 'Measure'] + df[group_col].map(str.upper).unique().tolist()
    return grouped

In [ ]:
desc_cat(df, "member_casual", cats)

In [ ]:
def pivot_cat(df, col1, col2):
    cros = pd.crosstab(df[col1], df[col2])
    cros = cros.rename_axis(None)
    cros = cros.rename_axis(None, axis=1)
    cros = cros.reset_index()
    cros = cros.rename(columns={'index': col1})
    return cros

In [ ]:
pivot_cat(df, "member_casual", "rideable_type")

**Visual Comparasion Of Numerical Fields Based On Membership (Graph 1)**

In [ ]:
def num_viz(df, col1, col2):
    hist = hv.Histogram(np.histogram(df[col2], 20))
    hist.opts(width=500, height=500, title=f'{col2} histogram', tools=['hover'])
    box = hv.BoxWhisker(df, kdims=[col1], vdims=[col2])
    box.opts(width=500, height=500, title=f'Range of {col2} by {col1}', tools=['hover'], box_color=col1, cmap='tab10')
    return pn.Row(hist, box)

In [ ]:
nums_col = pn.Column()
for num in nums:
    nums_col.append(num_viz(df, 'member_casual', num))
nums_col

**Visual Comparasion Of Categorical Fields Based On Membership (Graph 2)**

In [ ]:
def cat_viz(df, col1, col2):
    cmap = plt.get_cmap('tab10')
    colors = [cmap(i) for i in np.linspace(0, 1, 8)]
    pie = go.Figure(go.Pie(labels=df[col2].unique(), values=df[col2].value_counts(), marker=dict(colors=colors
                                                                , line=dict(color='#FFF', width=1)), insidetextorientation='radial'
                                                                , showlegend=False, textinfo='label+percent'))
    pie.update_layout(autosize=False, width=500, height=500, title_text=f'{col2} categories', margin=dict(
        l=10,
        r=100,
        b=100,
        t=100,
        pad=0))
    bars = hv.Bars(df, kdims=[col1, col2]).aggregate(function=np.size)
    hover = HoverTool(tooltips=[(col2, f"@{col2}")])
    bars.opts(stacked=True, width=500, height=500, title=f'{col2} by {col1}', tools=[hover], ylabel='Count', cmap='tab10')
    return pn.Row(pie, bars)

In [ ]:
cats_col = pn.Column()
for cat in chartable:
    cats_col.append(cat_viz(df, 'member_casual', cat))
cats_col

In [ ]:
ts = df.groupby(['date','member_casual'], as_index=False)['member_casual'].size()
member_counts = ts[ts['member_casual'] == 'member'][['date', 'size']]
member_counts["date"] = member_counts["date"].apply(pd.to_datetime)
casual_counts = ts[ts['member_casual'] == 'casual'][['date', 'size']]
casual_counts["date"] = casual_counts["date"].apply(pd.to_datetime)

**Trends of Rides For Members (Graph 3)**

In [ ]:
member_heat_cal = calplot(member_counts, x="date", y="size", title='Members rides by date', years_title=True, colorscale='reds',total_height=500,width=500)
mcurve = hv.Curve(member_counts, 'date', ('size', 'Count'))
mcurve.opts(width=500, height=500, title='Members rides by date', tools=['hover'], color='red')
trends_member = pn.Row(mcurve, member_heat_cal)
trends_member

**Trends of Rides for Casuals (Graph 4)**

In [ ]:
casual_heat_cal = calplot(casual_counts, x="date", y="size", title='Casuals rides by date', years_title=True, colorscale='blues',total_height=500,width=500)
ccurve = hv.Curve(casual_counts, 'date', ('size', 'Count'))
ccurve.opts(width=500, height=500, title='Casuals rides by date', tools=['hover'])
trends_casual = pn.Row(ccurve, casual_heat_cal)
trends_casual

In [ ]:
gdf = gpd.read_file(chicago_map, crs='epsg:4326')

In [ ]:
member_start_count = df[df['member_casual'] == 'member']
member_start_count = member_start_count.groupby(['start_suburb'], as_index=False)['start_suburb'].size()
map_member_start = gdf.merge(member_start_count, how='left', left_on='community', right_on='start_suburb')

In [ ]:
member_end_count = df[df['member_casual'] == 'member']
member_end_count = member_end_count.groupby(['end_suburb'], as_index=False)['end_suburb'].size()
map_member_end = gdf.merge(member_end_count, how='left', left_on='community', right_on='end_suburb')

In [ ]:
casual_start_count = df[df['member_casual'] == 'casual']
casual_start_count = casual_start_count.groupby(['start_suburb'], as_index=False)['start_suburb'].size()
map_casual_start = gdf.merge(casual_start_count, how='left', left_on='community', right_on='start_suburb')

In [ ]:
casual_end_count = df[df['member_casual'] == 'casual']
casual_end_count = casual_end_count.groupby(['end_suburb'], as_index=False)['end_suburb'].size()
map_casual_end = gdf.merge(casual_end_count, how='left', left_on='community', right_on='end_suburb')

**HeatMap Of Members Rides In Chicago Suburbs (Graph 5)**

In [ ]:
CHICAGO_COORDINATES = (41.85, -87.68)
mstart = folium.Map(location=CHICAGO_COORDINATES, width='100%', height='100%', zoom_start=10, tiles="cartodbpositron")
m_choropleth_start = folium.Choropleth(geo_data=map_member_start,
              data=map_member_start,
              columns=["start_suburb","size"],
              key_on="feature.properties.community",
              name="Member Rides Start Locations",
              legend_name="Start Trips Count",
              highlight=True,
              nan_fill_color='black',
              nan_fill_opacity=0.6,                  
              fill_color="YlOrRd").add_to(mstart)
m_choropleth_start.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=["community", "size"],
                                   aliases=['Suburb:', 'Number of Ries:'],
                                   labels=True,
                                   localize=True,
                                   sticky=False,
                                   style="""
                                   background-color: #F0EFEF;
                                   border: 2px solid black;
                                   border-radius: 3px;
                                   box-shadow: 3px;
                                   """,))
m_choropleth_end = folium.Choropleth(geo_data=map_member_end,
              data=map_member_end,
              columns=["end_suburb", "size"],
              key_on="feature.properties.community",
              name="Member Rides End Locations",
              legend_name="End Trips Count",
              highlight=True,
              fill_color="YlOrRd",
              show=False).add_to(mstart)
m_choropleth_end.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=["community", "size"],
                                   aliases=['Suburb:', 'Number of Ries:'],
                                   labels=True,
                                   localize=True,
                                   sticky=False,
                                   style="""
                                   background-color: #F0EFEF;
                                   border: 2px solid black;
                                   border-radius: 3px;
                                   box-shadow: 3px;
                                   """,))
title_text = 'Mambers Rides HeatMap'
instructions = 'kindly click on the icon placed on the upper right corner, to choose the informatin that will be displayed'
title_html = f'''
             <h3 align="center" style="font-size:12px"><b>{title_text}</b></h3>
             <h6 align="center" style="font-size:12px">{instructions}</h6>
             '''
mstart.get_root().html.add_child(folium.Element(title_html))
folium.LayerControl().add_to(mstart)
mstart

**HeatMap Of Casuals Rides In Chicago Suburbs (Graph 6)**

In [ ]:
CHICAGO_COORDINATES = (41.85, -87.68)
cstart = folium.Map(location=CHICAGO_COORDINATES, width='100%', height='100%', zoom_start=10, tiles="cartodbpositron")
c_choropleth_start = folium.Choropleth(geo_data=map_casual_start,
              data=map_casual_start,
              columns=["start_suburb", "size"],
              key_on="feature.properties.community",
              name="Casual Rides Start Locations",
              legend_name="Start Trips Count",
              highlight=True,
              fill_color="YlGnBu").add_to(cstart)
c_choropleth_start.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=["community", "size"],
                                   aliases=['Suburb:', 'Number of Ries:'],
                                   labels=True,
                                   localize=True,
                                   sticky=False,
                                   style="""
                                   background-color: #F0EFEF;
                                   border: 2px solid black;
                                   border-radius: 3px;
                                   box-shadow: 3px;
                                   """,))
c_choropleth_end = folium.Choropleth(geo_data=map_casual_end,
              data=map_casual_end,
              columns=["end_suburb", "size"],
              key_on="feature.properties.community",
              name="Casual Rides End Locations",
              legend_name="End Trips Count",
              highlight=True,
              fill_color="YlGnBu",
              show=False).add_to(cstart)
c_choropleth_end.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=["community", "size"],
                                   aliases=['Suburb:', 'Number of Ries:'],
                                   labels=True,
                                   localize=True,
                                   sticky=False,
                                   style="""
                                   background-color: #F0EFEF;
                                   border: 2px solid black;
                                   border-radius: 3px;
                                   box-shadow: 3px;
                                   """,))
title_text = 'Casuals Rides HeatMap'
instructions = 'kindly click on the icon placed on the upper right corner, to choose the informatin that will be displayed'
title_html = f'''
             <h3 align="center" style="font-size:12px"><b>{title_text}</b></h3>
             <h6 align="center" style="font-size:12px">{instructions}</h6>
             '''
cstart.get_root().html.add_child(folium.Element(title_html))
folium.LayerControl().add_to(cstart)
cstart

In [ ]:
log("Cyclistic Notebook Execution Ended")

### Key Findings

**Table 1 Key Findings:**
* Farthest distance traveled by both categories is 32km (20 Miles) and they have a similar average of 2km (1.25 Miles)
* Members tend to ride for shorter periods in general, nearly half the time when compared to casual rides
* Members ride faster than the casual customer most of the time

**Table 2 Key Findings:**
* Members prefer the classic bike, whereas the casual riders enjoy the options of bikes almost equally.
* Summer is the in-demand season for bikes while in winter bikes are less used significantly
* Members ride more on the weekend days while casual enjoy the rides any day of the week
* Afternoon is the period of the day with high usage of bikes for both categories, but members take almost twice the rides when compared to casuals during the morning time


**Graph 3 Key Findings:**
* Most of the rides are happening during the summer and fall for members

**Graph 4 Key Findings:**
* Most of the rides are happening during the summer and fall for casual riders too, with Saturday being with the most rides during the week notably

**Graph 5 Key Findings:**
* Near North Side Suburb is the most visited area in Chicago by members
* Lincoln Park and Lake View areas come as the second choice for the members
* Loop, Near West Side, and West Town are considering popular destinations for the members
* Members mostly do not ride to the outskirts of the city

**Graph 6 Key Findings:**
* Near North Side Suburb is the most visited area in Chicago by casuals too
* Loop and Lincoln Park areas come as the second choice for the casuals
* Lake View comes as the third choice for the casuals
* Near West Side and West Town are considered popular destinations for the casuals too
* Members mostly do not ride to the outskirts of the city too

### Recommendations

My three recommendations for the new marketing strategy are as follows:  
1. Time-Related Recommendation: ensuring the demand for bicycles are met and having availability also during peak times:
* Morning and afternoon for the day
* Weekends for the week
* Summer and fall for the year

2. Location Related Recommendation: 6 Chicago suburbs being the most visited by riders, in those suburbs where the offering of bikes and advertisement should take place:
* Near North Side
* Lincoln Park
* Lake View
* Loop
* Near West Side
* West Town

3. Equipment Related Recommendation: since the members tend to use the bicycles for shorters period and faster travel times, introducing more electric bikes with better mileage and higher speed for members will keep the current members happy and spark interest for casual who need quick rides, not to forget it is less physical,environment friendly and also can help in having more riders to explore the other suburbs of the city

# Slides Section

## Motivate Inc. Customers: Deeper Understanding

**Created by: Adel Ahmed**

**Jan 2022**

### OUTLINE ###
1. Executive Summary
2. Introduction
3. Methodology
4. Results
    - Visualization – Charts
5. Discussion
6. Findings & Implications
7. Conclusion
8. Appendix

##### Section 1 #####

### EXECUTIVE SUMMARY ###
- Identify how annual members and casual riders differ
- Recommendations for the new marketing strategy based on findings

##### Section 2 #####

### INTRODUCTION ###
The head of Marketing believes that maximizing the number of annual members will be key to future growth.  
Moreno has set a clear goal: Design marketing strategies aimed at converting casual riders into annual members.

Marketing team are interested in analyzing the Cyclistic historical bike trip data to identify trends  
The question to answer in this Analysis is: **How do annual members and casual riders use Cyclistic bikes differently?**  
Also recommendations will be provided based on the finding of this Analysis


Description of the dataset

| No. | Column | Description| Data Type | Category|
| :-- | :-- | :-- | :-- | :-- |
| 1 | Ride_Id | Customer Unique ID | String | Nominal |
| 2 | Rideable_Type | The type of bike customer used for the trip | String | Nominal |
| 3 | Started_At | The date and time when the trip started | DateTime | Discrete |
| 4 | Ended_At | The date and time when the trip ended | DateTime | Discrete |
| 5 | Start_Station_Name | Name of the station where the trip started | String | Nominal |
| 6 | Start_Station_Id | ID of the station where the trip started | String | Nominal |
| 7 | End_Station_Name | Name of the station where the trip ended | String | Nominal |
| 8 | End_Station_Id | ID of the station where the trip ended | String | Nominal |
| 9 | Start_Lat | Latitude value where the trip have started expressed in decimal degrees | Float | Contiuous |
| 10 | Start_Lng | Longitude value where the trip have started expressed in decimal degrees | Float | Contiuous |
| 11 | End_Lat | Latitude value where the trip have ended expressed in decimal degrees | Float | Contiuous |
| 12 | End_Lng | Longitude value where the trip have ended expressed in decimal degrees | Float | Contiuous |
| 13 | Member_Casual | Classification of Customers: Customers who purchase single-ride or full-day passes are referred to as casual. Customers who purchase annual memberships referred to as members  | String | Nominal |

##### Section 3 #####

### METHODOLOGY ###
- Collecting Data from the publicly avaialbe dataset
- Acquring location data from publicly available API
- Examining all data to conduct the analysis
- Exploring the data through statistics and visualization
- Listing the findings and insights from the analysis
- Recommendations for the Marketing Team


##### Section 4 #####

### RESULTS ###

* Exploratory Data Analysis Results


#### Exploratory Data Analysis Results ####

In [ ]:
hv.extension('bokeh', 'plotly', logo=False)

In [ ]:
num_viz(df, 'member_casual', 'ride_length_mins')

In [ ]:
cat_viz(df, 'member_casual', 'season')

##### Section 5 #####

### DISCUSSION ###

Members drive faster on average while casual riders tend to ride for longer periods of time

##### Section 6 #####

### Findings & Implications ###
1. Time-Related Recommendation: ensuring the demand for bicycles are met and having availability also during peak times:
    * Morning and afternoon for the day
    * Weekends for the week
    * Summer and fall for the year

2. Location Related Recommendation: 6 Chicago suburbs being the most visited by riders, in those suburbs where the offering of bikes and advertisement should take place:
    * Near North Side
    * Lincoln Park
    * Lake View
    * Loop
    * Near West Side
    * West Town

3. Equipment Related Recommendation: since the members tend to use the bicycles for shorters period and faster travel times, introducing more electric bikes with better mileage and higher speed for members will keep the current members happy and spark interest for casual who need quick rides, not to forget it is less physical,environment friendly and also can help in having more riders to explore the other suburbs of the city

##### Section 7 #####

### CONCLUSION ###
Summer campaign targeted at high demand areas with new and faster bycicles options should be the new strategy for marketing.

##### Section 8 #####

### APPENDIX ###

More information can be found at [My Portfolio](https://panel-adahmed.herokuapp.com/)

# Thank you #
Looking forward to seeing you agian.

# Dashboard Section

In [ ]:
html = f"<iframe src={slides_file}, width=1320, height=520></iframe>"
html_pane = pn.pane.HTML(html, sizing_mode='scale_width')

In [ ]:
dashboard = pn.GridSpec(width=1300, height=700)
sidebar = dashboard[:, 0:2] = pn.Column(margin=0, sizing_mode='scale_height')
header = dashboard[0:1, 2:12] = pn.Row(margin=0, sizing_mode='scale_width')
main = dashboard[1:6, 2:12] = pn.FlexBox(margin=0, sizing_mode='scale_width', align_content='flex-start')


x = pn.widgets.Select(name='X-Axis', value='ride_length_mins', options=nums)
y = pn.widgets.Select(name='Y-Axis', value='ride_length_mins', options=nums)
z = pn.widgets.Select(name='Z-Axis', value='date', options=dates)
size = pn.widgets.Select(name='Size', value='None', options=['None'] + nums)
color = pn.widgets.Select(name='Color', value='None', options=['None'] + chartable)

@pn.depends(x.param.value, y.param.value, color.param.value, size.param.value) 
def create_figure(x, y, color, size):
    opts = dict(cmap='rainbow', width=500, height=500, line_color='black')
    if color != 'None':
        opts['color'] = color 
    if size != 'None':
        opts['size'] = hv.dim(size).norm()*20
    return hv.Points(df, [x, y], label="%s vs %s" % (x.title(), y.title())).opts(**opts)


@pn.depends(x.param.value, z.param.value) 
def create_trends(x, z):
    opts = dict(width=500, height=400, color='red', xaxis=None, tools=['hover'])
    curve = hv.Curve(df.sort_values(by=[z]), z, x, label="%s Trend" % (x.title())).opts(**opts)
    spike = hv.Spikes(df[x]).opts(width=500, height=100, line_alpha=0.2, yaxis=None, color='grey')
    stacked = (curve + spike).cols(1)
    return stacked


widgets = pn.WidgetBox(x, y, z , color, size)


title = '## Motivate Inc. Customers: Deeper Understanding'
introduction = 'Dashboard based on the dataset of cycling trips in Chicago City from the month of Aug 2020 until the Jul 2021'
text = pn.Column(pn.pane.Markdown(title, sizing_mode='scale_width'), pn.pane.Markdown(introduction, sizing_mode='scale_width'), sizing_mode='scale_width')
logo = pn.pane.JPG('https://media-exp1.licdn.com/dms/image/C4D0BAQFg_eRgwGXdKw/company-logo_200_200/0/1520880022675?e=2159024400&v=beta&t=Rm-oye7hhlq0lbT0H5z1gRco0aV_YwOBamyPtDrLQvk', width=100,  height=100)

sidebar.append(widgets)
header.extend([text, pn.layout.Spacer(width=10), logo])
main.append(create_figure)
main.append(create_trends)
# main.append(trends_member)
# main.append(trends_casual)

In [ ]:
tech_report = pn.Column()

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('### Introduction')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown("""In 2016, Cyclistic launched a successful bike-share offering. Since then, the program has grown to a fleet of 5,824 bicycles that are geotracked and locked into a network of 692 stations across Chicago. The bikes can be unlocked from one station and returned to any other station in the system anytime. Cyclistic sets itself apart by also offering reclining bikes, hand tricycles, and cargo bikes, making bike-share more inclusive to people with disabilities and riders who can’t use a standard two-wheeled bike. The majority of riders opt for traditional bikes; about 8% of riders use the assistive options. Cyclistic users are more likely to ride for leisure, but about 30% use them to commute to work each day.

Until now, Cyclistic’s marketing strategy relied on building general awareness and appealing to broad consumer segments. One approach that helped make these things possible was the flexibility of its pricing plans: single-ride passes, full-day passes, and annual memberships. Customers who purchase single-ride or full-day passes are referred to as casual riders. Customers who purchase annual memberships are Cyclistic members. Cyclistic’s finance analysts have concluded that annual members are much more profitable than casual riders. Although the pricing flexibility helps Cyclistic attract more customers, The Director of Marketing Moreno believes that maximizing the number of annual members will be key to future growth. Rather than creating a marketing campaign that targets all-new customers, Moreno believes there is a very good chance to convert casual riders into members. She notes that casual riders are already aware of the Cyclistic program and have chosen Cyclistic for their mobility needs.

Moreno has set a clear goal: Design marketing strategies aimed at converting casual riders into annual members. In order to do that, however, the marketing analyst team needs to better understand:

1- how annual members and casual riders differ

2- why casual riders would buy a membership

3- how digital media could affect their marketing tactics

Moreno and her team are interested in analyzing the Cyclistic historical bike trip data to identify trends.
Your are a junior data analyst working in the marketing analyst team. **Moreno has assigned you the first question to answer: How do annual members and casual riders use Cyclistic bikes differently?** To answer that, you are going to conduct this data analysis. From the analysis insights, marketing team will design a new marketing strategy to convert casual riders into annual members. But first, Cyclistic executives must approve marketing team recommendations, so the analysis must be backed up with compelling data insights and professional data visualizations
""", sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown("""### Table of Contents  
<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#section1">Main Objective</a><br>
2. <a href="#section2">Data Acquisition</a><br>
3. <a href="#section3">Data Processing</a><br>
4. <a href="#section4">Exploratory Data Analysis</a><br>
5. <a href="#section5">Key Findings</a><br>
6. <a href="#section6">Recommendations</a><br>

</font>
</div>
""", sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown("""###Main Objective</h3>

| Item |Defintion|
| :-- | :-- | 
| Category Of Data Analysis | Descriptive | 
| Business Task Need To Be Addressed| Identify how annual members and casual riders use cyclistic bikes differently | 
| Goal Of The Business Task | Help marketing team in designing marketing strategies aimed at converting casual riders into annual members | 
| KPI | Members Conversion Rate | 
| Key Stakeholders | Cyclistic Executive Team, Director of Marketing (Lily Moreno) and Marketing Analytics Team | 
""", sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown("""###Data Acquisition</h3>


Cyclistic’s historical trip data has been made available by [Motivate International Inc](https://www.motivateco.com/). under this [license](https://www.divvybikes.com/data-license-agreement)

[Data Source](https://divvy-tripdata.s3.amazonaws.com/index.html)

The portion of the data that is required to be analyzed is the previous 12 months of Cyclistic trip data Data. the information resides in ZIP files, so they need extracting first. the resulting is 12 CSV files format with 13 columns in each one,

| No. | Column | Description| Data Type | Category|
| :-- | :-- | :-- | :-- | :-- |
| 1 | Ride_Id | Customer Unique ID | String | Nominal |
| 2 | Rideable_Type | The type of bike customer used for the trip | String | Nominal |
| 3 | Started_At | The date and time when the trip started | DateTime | Discrete |
| 4 | Ended_At | The date and time when the trip ended | DateTime | Discrete |
| 5 | Start_Station_Name | Name of the station where the trip started | String | Nominal |
| 6 | Start_Station_Id | ID of the station where the trip started | String | Nominal |
| 7 | End_Station_Name | Name of the station where the trip ended | String | Nominal |
| 8 | End_Station_Id | ID of the station where the trip ended | String | Nominal |
| 9 | Start_Lat | Latitude value where the trip have started expressed in decimal degrees | Float | Contiuous |
| 10 | Start_Lng | Longitude value where the trip have started expressed in decimal degrees | Float | Contiuous |
| 11 | End_Lat | Latitude value where the trip have ended expressed in decimal degrees | Float | Contiuous |
| 12 | End_Lng | Longitude value where the trip have ended expressed in decimal degrees | Float | Contiuous |
| 13 | Member_Casual | Classification of Customers: Customers who purchase single-ride or full-day passes are referred to as casual. Customers who purchase annual memberships referred to as members  | String | Nominal |




Since the dataset is made public by a known corporation and has a license of how we can use it, we can safely say the data is credible.  
The Longitude and latitude information have been extracted, enriched with Suburb information, and provided as separate CSV files to join them with original data later on.  
Also, map file will be downloaded from the [City of Chigaco Website](https://data.cityofchicago.org/api/geospatial/cauq-8yn6?method=export&format=GeoJSON), which will help in producing geographic visualization
""", sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('###Data Processing</h3>', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown("""
Tools Used For The Analysis  
Python langauge, SQL(Postgres) and NoSQL(MongoDB) will be the used for this analysis
""", sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown("""
Libraries Used For The Analysis:  
1. numpy  
2. pandas  
3. plotly_calplot  
4. datetime  
5. tqdm  
6. funcy  
7. configparser  
8. asyncio  
9. asyncpg  
10. aiosql  
11. geopandas  
12. geopy  
13. shapely  
14. folium  
15. holoviews  
16. panel  
""", sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown("""
Data and Software Engineering practices have been implemented during this analysis:  
1. Implementing ETL process with 3 functions representing each step  
2. Encapsulation of Sensitive information by offloading it into configuration files and access the informarion as varialbes  
3. Code seperation by combining all the SQL code into external file and access them as functions that can be called  
4. Logging of the process to keep track of the functions called and the time of execution  
5. Clear structure by keep data, configuration, logs into seperate folders  
6. Converting code into function to allow testing and docstring provided to describe the code  
Complete code can be obtain at the Data tab as downloadable notebook
""", sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown("""
```python
log("Data Extract Phase Started")
df = extract(sample_data)
log("Data Extract Phase Ended")
log("Data Transform Phase Started")
df = transform(df)
log("Data Transform Phase Ended")
log("Data Load Phase Started")
load(df, processed_df_path)
log("Data Load Phase Ended")
```
""", sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('###Exploratory Data Analysis</h3>', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('**Descriptive Analysis Of Numerical Fields (Table 1)**', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.pane.DataFrame(desc_num(df, "member_casual", nums).head(10), sizing_mode='scale_width'))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('**Descriptive Analysis Of Categorical Fields (Table 2)**', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.pane.DataFrame(desc_cat(df, "member_casual", cats).head(4), sizing_mode='scale_width'))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('**Visual Comparasion Of Numerical Fields Based On Membership (Graph 1)**', sizing_mode='scale_width')))

In [ ]:
tech_report.append(nums_col)

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('**Visual Comparasion Of Categorical Fields Based On Membership (Graph 2)**', sizing_mode='scale_width')))

In [ ]:
tech_report.append(cats_col)

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('**Trends of Rides For Members (Graph 3)**', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(mcurve, member_heat_cal, sizing_mode='scale_width'))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('**Trends of Rides for Casuals (Graph 4)**', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(ccurve, casual_heat_cal,sizing_mode='scale_width'))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('**HeatMap Of Members Rides In Chicago Suburbs (Graph 5)**', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.panel(mstart, height = 600, sizing_mode='scale_both'))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('**HeatMap Of Casuals Rides In Chicago Suburbs (Graph 6)**', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.panel(cstart,  height = 600, sizing_mode='scale_both'))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('### Key Findings', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('''
**Table 1 Key Findings:**  
    - Farthest distance traveled by both categories is 32km (20 Miles) and they have a similar average of 2km (1.25 Miles)  
    - Members tend to ride for shorter periods in general, nearly half the time when compared to casual rides  
    - Members ride faster than the casual customer most of the time

**Table 2 Key Findings:**  
    - Members prefer the classic bike, whereas the casual riders enjoy the options of bikes almost equally.  
    - Summer is the in-demand season for bikes while in winter bikes are less used significantly  
    - Members ride more on the weekend days while casual enjoy the rides any day of the week  
    - Afternoon is the period of the day with high usage of bikes for both categories, but members take almost twice the rides when compared to casuals during the morning time

**Graph 3 Key Findings:**  
    - Most of the rides are happening during the summer and fall for members

**Graph 4 Key Findings:**  
    - Most of the rides are happening during the summer and fall for casual riders too, with Saturday being with the most rides during the week notably

**Graph 5 Key Findings:**  
    - Near North Side Suburb is the most visited area in Chicago by members  
    - Lincoln Park and Lake View areas come as the second choice for the members  
    - Loop, Near West Side, and West Town are considering popular destinations for the members  
    - Members mostly do not ride to the outskirts of the city

**Graph 6 Key Findings:**  
    - Near North Side Suburb is the most visited area in Chicago by casuals too  
    - Loop and Lincoln Park areas come as the second choice for the casuals  
    - Lake View comes as the third choice for the casuals  
    - Near West Side and West Town are considered popular destinations for the casuals too  
    - Members mostly do not ride to the outskirts of the city too  
''', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('### Recommendations', sizing_mode='scale_width')))

In [ ]:
tech_report.append(pn.Row(pn.pane.Markdown('''
My three recommendations for the new marketing strategy are as follows:  
1. Time-Related Recommendation: ensuring the demand for bicycles are met and having availability also during peak times:  
    - Morning and afternoon for the day  
    - Weekends for the week  
    - Summer and fall for the year  
2. Location Related Recommendation: 6 Chicago suburbs being the most visited by riders, in those suburbs where the offering of bikes and advertisement should take place:  
    - Near North Side  
    - Lincoln Park  
    - Lake View  
    - Loop  
    - Near West Side  
    - West Town  
3. Equipment Related Recommendation: since the members tend to use the bicycles for shorters period and faster travel times, introducing more electric bikes with better mileage and higher speed for members will keep the current members happy and spark interest for casual who need quick rides, not to forget it is less physical,environment friendly and also can help in having more riders to explore the other suburbs of the city
''', sizing_mode='scale_width')))

In [ ]:
data_files = pn.Column()

In [ ]:
data_files.append(pn.widgets.FileDownload(file=processed_df_path, filename='data.csv'))
data_files.append(pn.widgets.FileDownload(file=sql_path, filename='queris.sql'))
data_files.append(pn.widgets.FileDownload(file=chicago_map, filename='chicago_map.geojson'))
data_files.append(pn.widgets.FileDownload(file=slides_file, filename='slides.html'))

In [ ]:
code_preview = pn.Column()
code_preview.append(pn.widgets.FileDownload(file='apps/Data_Analysis.ipynb', filename='Code.ipynb'))

In [ ]:
layout = pn.Tabs(('Summary Report', html_pane), ('Dashboard', dashboard),
                 ('Technical Report', tech_report), ('Code', code_preview),
                 ('Data', data_files), sizing_mode='scale_width')
layout.servable()